In [ ]:
# Premonitor: utils.py
# This file contains helper functions used across the project, primarily for
# data loading, pre-processing, and augmentation.

import os
import numpy as np
import tensorflow as tf
import librosa
from tqdm import tqdm
import glob

# Import our custom project configuration
import config

# --- Audio Processing Utilities ---

def audio_to_spectrogram(audio_path):
    """
    Loads a .wav file and converts it into a log-mel spectrogram.
    This turns an audio signal into an image-like representation that a CNN can process.
    """
    try:
        y, sr = librosa.load(audio_path, sr=config.SPECTROGRAM_SAMPLE_RATE)
        mel_spectrogram = librosa.feature.melspectrogram(
            y=y, sr=sr, n_mels=config.SPECTROGRAM_N_MELS, hop_length=config.SPECTROGRAM_HOP_LENGTH
        )
        log_mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
        return log_mel_spectrogram
    except Exception as e:
        if config.DEBUG_MODE:
            print(f"Error processing audio file {audio_path}: {e}")
        return None

def create_acoustic_dataset(data_dir, environmental_dirs=[]):
    """
    Walks through dataset directories, loads audio files, converts them to spectrograms,
    and creates training pairs.

    For machine sounds (data_dir): Creates (normal_spec, 0) and (flipped_spec, 1) pairs.
    For environmental sounds: Creates (env_spec, 2) pairs to be ignored.

    Args:
        data_dir (str): Path to the primary machine sound dataset (e.g., MIMII).
        environmental_dirs (list): List of paths to environmental sound datasets (e.g., Urbansound8K).

    Returns:
        A tuple (spectrograms, labels) ready for training.
    """
    print("Creating acoustic dataset...")
    spectrograms = []
    labels = []

    # Process primary machine sounds for normal/anomaly training
    normal_dir = os.path.join(data_dir, 'train', 'normal')
    if not os.path.exists(normal_dir):
        print(f"Error: Directory not found - {normal_dir}")
    else:
        normal_files = [os.path.join(normal_dir, f) for f in os.listdir(normal_dir) if f.endswith('.wav')]
        for audio_file in tqdm(normal_files, desc="Processing normal machine audio"):
            spec = audio_to_spectrogram(audio_file)
            if spec is not None:
                spec_resized = tf.image.resize(spec[..., np.newaxis], config.ACOUSTIC_MODEL_INPUT_SHAPE[:2])
                spectrograms.append(spec_resized)
                labels.append(0) # Label 0 for 'normal'
                spectrograms.append(tf.image.flip_up_down(spec_resized))
                labels.append(1) # Label 1 for 'pseudo-anomaly'

    # Process environmental sounds to be ignored
    for env_dir in environmental_dirs:
        if not os.path.exists(env_dir):
            print(f"Warning: Environmental sound directory not found - {env_dir}")
            continue
        env_files = glob.glob(os.path.join(env_dir, '**', '*.wav'), recursive=True)
        for audio_file in tqdm(env_files, desc=f"Processing environmental audio from {os.path.basename(env_dir)}"):
            spec = audio_to_spectrogram(audio_file)
            if spec is not None:
                spec_resized = tf.image.resize(spec[..., np.newaxis], config.ACOUSTIC_MODEL_INPUT_SHAPE[:2])
                spectrograms.append(spec_resized)
                labels.append(2) # Label 2 for 'environmental/ignore'

    # NOTE: For very large datasets, loading all data into a NumPy array can consume
    # a lot of RAM. A more advanced approach would be to use tf.data.Dataset.from_generator.
    # For the MVP, this approach is sufficient and simpler to debug.
    return np.array(spectrograms), np.array(labels)


# --- Image Processing Utilities ---

def load_all_thermal_image_paths(dataset_dirs):
    """Scans multiple directories and returns a list of all image file paths."""
    all_paths = []
    for data_dir in dataset_dirs:
        if not os.path.exists(data_dir):
            print(f"Warning: Thermal dataset directory not found at '{data_dir}'. Skipping.")
            continue
        # Use glob to find all image files recursively
        image_files = glob.glob(os.path.join(data_dir, '**', '*.jpeg'), recursive=True)
        image_files.extend(glob.glob(os.path.join(data_dir, '**', '*.jpg'), recursive=True))
        image_files.extend(glob.glob(os.path.join(data_dir, '**', '*.png'), recursive=True))
        all_paths.extend(image_files)
    print(f"Found a total of {len(all_paths)} thermal images for pre-training.")
    return all_paths

def load_image_and_label(image_path, label):
    """Loads a single image and returns it with its label."""
    img = tf.io.read_file(image_path)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.resize(img, config.THERMAL_MODEL_INPUT_SHAPE[:2])
    img = tf.cast(img, tf.float32) / 255.0
    return img, label

def load_labeled_thermal_data(data_dir, batch_size, validation_split=0.2):
    """
    Loads a labeled dataset for fine-tuning, structured as:
    - data_dir/normal/*.jpg
    - data_dir/anomaly/*.jpg
    """
    if not os.path.exists(data_dir):
        print(f"Error: Labeled data directory not found at '{data_dir}'.")
        return None, None

    # Create a tf.data.Dataset from the directory structure
    train_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        labels='inferred',
        label_mode='binary', # For normal vs anomaly
        image_size=config.THERMAL_MODEL_INPUT_SHAPE[:2],
        batch_size=batch_size,
        validation_split=validation_split,
        subset='training',
        seed=123
    )
    val_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        labels='inferred',
        label_mode='binary',
        image_size=config.THERMAL_MODEL_INPUT_SHAPE[:2],
        batch_size=batch_size,
        validation_split=validation_split,
        subset='validation',
        seed=123
    )

    # Normalize the images
    normalization_layer = layers.Rescaling(1./255)
    train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
    val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

    return train_ds.prefetch(tf.data.AUTOTUNE), val_ds.prefetch(tf.data.AUTOTUNE)


@tf.function
def augment_image(image):
    """Applies a set of random augmentations to an image for SimSiam training."""
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_crop(image, size=[int(config.THERMAL_MODEL_INPUT_SHAPE[0]*0.8), int(config.THERMAL_MODEL_INPUT_SHAPE[1]*0.8), 3])
    image = tf.image.resize(image, size=config.THERMAL_MODEL_INPUT_SHAPE[:2])
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.clip_by_value(image, 0.0, 1.0)
    return image

def create_thermal_dataset_generator(image_paths, batch_size):
    """Creates a memory-efficient TensorFlow data generator for the SimSiam model."""
    path_ds = tf.data.Dataset.from_tensor_slices(image_paths)

    def load_for_siam(path):
        img = tf.io.read_file(path)
        img = tf.image.decode_image(img, channels=3, expand_animations=False)
        img = tf.image.resize(img, config.THERMAL_MODEL_INPUT_SHAPE[:2])
        return tf.cast(img, tf.float32) / 255.0

    image_ds = path_ds.map(load_for_siam, num_parallel_calls=tf.data.AUTOTUNE)

    # Create two augmented views for each image
    view1_ds = image_ds.map(augment_image, num_parallel_calls=tf.data.AUTOTUNE)
    view2_ds = image_ds.map(augment_image, num_parallel_calls=tf.data.AUTOTUNE)

    paired_ds = tf.data.Dataset.zip((view1_ds, view2_ds))

    return paired_ds.shuffle(1024).batch(batch_size).prefetch(tf.data.AUTOTUNE)